<a href="https://colab.research.google.com/github/murilosenai/Estudo-de-Python/blob/master/avaliacao2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação
Ambiente


In [ ]:
# Para instalar no colab:
!pip install -q dash
!pip install -q flask
!pip install -q dash_core_components
!pip install -q dash_html_components
!pip install -q dash_table
!pip install flask_ngrok

In [ ]:
# Instalar também o ngrok para rodar no background:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip 

In [ ]:
def start_ngrok():
  get_ipython().system_raw( './ngrok http 8050 &' ) 
  
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Rota index


In [ ]:
%%writefile dash/templates/index.html
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8"/>
    <title>dash com flash </title>
    <style>
    p {
        margin-bottom: 20 }
        p + p {
            text-indent: 3.5em;
            margin-top: 20 }
    p { text-align: right; }
    p { text-align: justify; }
    p { text-align: left; }
    
    div { text-align: center; }
    body {
        background-color: #fefbd8;
    }
    h1 {
        background-color: #80ced6;
        }
    div {
        background-color: #d5f4e6;
         } 
    ol li {
        background: #ffe5e5;
        padding: 5px;
        margin-left: 35px;
        }

        ul li {
            background: #cce5ff;
            margin: 5px;
        } 

    a:link, a:visited {
	text-decoration: none
	}
a:hover {
	text-decoration: underline; 
	color: #f00
	}
a:active {
	text-decoration: none
	}
    
    </style>
</head>
<body>

  <div>
    <h1> Introdução ao tema do dataset Titanic</h1>
       <p></p><p>O RMS Titanic foi um navio de passageiros britânico que começou a ser construído em 1909.
          Foi construído para ser o mais pesado e mais luxuoso navio da época, gerando lendas de que era supostamente
          “infundável”.</p>

       <p></p><p>Sua viagem inaugural foi em 10 de Abril de 1912 e partiu de Southampton (Reino Unido) com destino a Nova York
       (Estados Unidos), passando por Cherbourg-Octeville (França) e Queenstown na Irlanda. Em 14 de Abril às 23:40
        ele colidiu com um iceberg e afundou na madrugada do dia seguinte com mais de 1.500 pessoas a bordo, sendo considerado 
        um dos maiores naufrágios da história. Menos de um terço daqueles que estavam a bordo do Titanic sobreviveram.
         Os motivos para o naufrágio foram diversas causas, tanto naturais quanto humanas.
          Este acontecimento expôs diversas fragilidades levando a mudanças nas leis internacionais de navegação.</p> 

         <p><h2> O RMS Titanic¶</h2><p/>

          <p> O Titanic afundou no Oceano Atlântico Norte, nas primeiras horas da manhã de 15 de abril de 1912, após colidir com um iceberg 
          durante sua viagem inaugural de Southampton para a cidade de Nova York. Havia um número estimado de 2.224 passageiros e tripulantes a bordo do navio,
          e mais de 1.500 morreram, tornando-se um dos mais mortais desastres marítimos comerciais em tempos de paz da história moderna. 
          O RMS Titanic foi o maior navio à tona na época em que entrou em operação e foi o segundo de três transatlânticos de classe olímpica operados pela White Star Line. O Titanic foi construído pelo estaleiro Harland and Wolff em Belfast. 
          Thomas Andrews, seu arquiteto, morreu no desastre. <a href="https://www.kaggle.com/samukaunt/titanic-passo-a-passo-com-8-modelos-ml-pt-br/notebook" target="_blank">Para saber Mais</a></p>

        <h2>Seguem algumas perguntas a serem investigadas:</h2> 
        <p><p/><p>Os passageiros das primeiras classes tiveram maiores chances de sobrevivência?
          As mulheres tiveram maior chance de sobrevivência que os homens?
          as mulheres da primeira classe tiveram maior chance de sobrevivência que as mulheres da terceira classe?
          Crianças tiveram uma taxa maior de sobrevivência? Crianças da primeira e segunda classe sobreviveram mais que
          as crianças da terceira classe?</p>
      </div>
  <center>
    <ul>
    <h1> Rotas e funcionalidades: </h1>
    <li><h2> <a href="dados"> 2 – Rota Dados:  </a></h2></li>
    <li><h2> <a href="filtro"> 3 – Rota Filtro:  </a></h2></li>
    <li><h2> <a href="grafico"> 4 – Rota Grafico: </a></h2></li>
    {{arquivo}}
  </center> 
</ul>
</body>
</html>

In [ ]:
%%writefile dash/templates/grafico.html
<!DOCTYPE html>
{% block title %} grafico {% endblock %}
<html>
<head>
    <meta charset="UTF-8"/>
    {% block title %} grafico {% endblock %}
<style>
    a:link, a:visited {
        text-decoration: none
        }
        a:hover {
            text-decoration: underline; 
            color: #f00
            }
            a:active {
                text-decoration: none
                }
</style>
</head>
<body>
<a href="javascript: history.go(-1)">Voltar a Pagina Principal</a> 

</body>
{% block title %} grafico {% endblock %}
</html>

In [ ]:
%%writefile dash/main.py

import flask
import dash
import dash_core_components as dcc
import dash_html_components as html
from flask import Flask, render_template
from dash.dependencies import Input, Output
import pandas as pd

import random

server = flask.Flask(__name__)
################################### index ============================
@server.route('/')
@server.route("/index/")
def index():
  return render_template('index.html')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

################################### tabela ============================
app = dash.Dash(
    __name__,
    server=server,
    routes_pathname_prefix='/dados/'
)

def generate_table(dataframe, max_rows=10):
  return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )
  
app.layout = html.Div(children=[
    html.H4(children='Titanic Embarcação'),
    html.A('Voltar a Pagina Principal', href='javascript: history.go(-1)'),
    html.Br(),
    html.Br(),
    generate_table(df)       
])

def update_output_div(input_value):
  return 'You\'ve entered "{}"'.format(input_value)


################################### Grafico ============================
app = dash.Dash(
    __name__,
    server=server,
    routes_pathname_prefix='/grafico/'
)
app.layout = html.Div(children=[
                                html.H1(children='Ola Dash'),
                                html.A('Voltar a Pagina Principal', href='javascript: history.go(-1)'),
                                html.Br(),
                                html.Div(children='''
                                NÃO consegui fazer, tentei com tres graficos diferentes, mas o que estava dando certo, não deu mais tempo de acabar.
                                abaixo no cogigo fonte, esta os codigos de tentativa como testes.
                                '''),

  dcc.Graph(
      id='example',
      figure={
          'data': [
                   {'x': [1, 2, 3, 4, 5], 'y': [9, 6, 2, 1, 5], 'type': 'line', 'name': 'Boats'},
                   {'x': [1, 2, 3, 4, 5], 'y': [8, 7, 2, 7, 3], 'type': 'bar', 'name': 'Cars'},
                   ],
              'layout': {
                  'title': 'Basic Dash Example'                   
                  }
              }
            )
   
  ])

def update_output_div(input_value):
  return 'You\'ve entered "{}"'.format(input_value)



################################### filtro ============================


app = dash.Dash(
    __name__,
    server=server,
    routes_pathname_prefix='/filtro/'
    )

app.layout = html.Div([
                       dcc.Dropdown(
                           id='demo-dropdown',
                           options=[
                                    {'label': 'Titanic Escolha a embarcação:', 'value': 'SF'},
                                    {'label': 'Titanic Embarcação de Homens:', 'value': 'NYC'},
                                    {'label': 'Titanic Embarcação de Mulheres:', 'value': 'MTL'},
                                    {'label': 'Titanic Embarcação de Crianças:', 'value': 'SFR'}
                                    ],
                                    value='SF'
                                  ),
                       html.H4(children='Titanic Embarcação'),
                       html.A('Voltar A Pagina Principal', href='javascript: history.go(-1)'),
                       html.Br(),
                       html.Br(),                       
                       html.Div(id='dd-output-container')
                       ])
@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
  return 'You have selected "{}"'.format(value)
  

if __name__ == '__main__':
  app.run_server(debug=True)


Overwriting dash/main.py


In [ ]:
start_ngrok()
!python dash/main.py

Testes para a prova


In [ ]:
%%writefile dash/graph.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
mgr_options = df["Sex"].unique()

app = dash.Dash()

app.layout = html.Div([
    html.H2("Titanic Relatorio"),
    html.Div(
        [
            dcc.Dropdown(
                id="Sex",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Sexs'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])


@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Sex', 'value')])
def update_graph(Manager):
    if Sex == "All Sexs":
        df_plot = df.copy()
    else:
        df_plot = df[df['Sex'] == Sex]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Pclass"],
        values=['Age'],
        aggfunc=sum,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('Age', '1')], name='Primeira Classe')
    trace2 = go.Bar(x=pv.index, y=pv[('Age', '2')], name='Segunda Classe ')
    trace3 = go.Bar(x=pv.index, y=pv[('Age', '1')], name='Terceira Classe')
    

    return {
        'data': [trace1, trace2, trace3],
        'layout':
        go.Layout(
            title='Titanic Relatorio {}'.format(Sex),
            barmode='stack')
    }


if __name__ == '__main__':
    app.run_server(debug=True)

Writing dash/graph.py


In [ ]:

start_ngrok()
!python dash/graph.py 

In [ ]:
# %%writefile dash/graficos.py
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
import warnings
warnings.warn = False

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

gender_submission = pd.read_csv("https://raw.githubusercontent.com/minsuk-heo/kaggle-titanic/master/input/gender_submission.csv")
test = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
train = pd.read_csv("https://raw.githubusercontent.com/minsuk-heo/kaggle-titanic/master/input/train.csv")

print("Training set = ",train.shape)
print("Testing set = ",test.shape)
print("Sum of Missing Values (Train/Test)= ",train.isna().sum().sum(),"(",test.isna().sum().sum(),")")
print("Survival Rate (in Training Data) =",round(train.Survived.sum()/train.shape[0]*100,2),"%")
print("Missing Values in Training Dataset:\n",round(train.isna().sum()/train.shape[0]*100,2))
print("Missing Values in Testing Dataset:\n",round(test.isna().sum()/test.shape[0]*100,2))
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error


def get_accuracy_score(data,col):
  X_train, X_test, y_train, y_test = train_test_split(data[col], data['Age'], test_size=0.3, random_state=7)

  lr1=LinearRegression()
  lr1.fit(X_train,y_train)
  y_pred = lr1.predict(X_test)
  return mean_squared_error(y_test, y_pred)
def get_missing_age(data):
  index = np.where(data.Age.isnull())
  
  # Find Prediction based on Present Elements
  data1=data[~data.Age.isnull()]
  acc_score0 = get_accuracy_score(data1,['SibSp','Parch','Fare'])
  acc_score1 = get_accuracy_score(data1,['SibSp','Parch'])
  acc_score2 = get_accuracy_score(data1,['SibSp','Fare'])
  acc_score3 = get_accuracy_score(data1,['Parch','Fare'])
  min_score = min(acc_score0,acc_score1,acc_score2,acc_score3) # Error to be minimized

  # Fit Model by Best Feature Selection
  data2 = data[data.Age.isnull()] 
  if min_score == acc_score0:
    col = ['SibSp','Parch','Fare']  
  else:
    if min_score == acc_score1:
      col = ['SibSp','Parch']
    else:
      if min_score == acc_score2:
        col = ['SibSp','Fare']                
      else:
        col = ['Parch','Fare']        
  X_train, y_train, X_test = data1[col], data1['Age'], data2[col]    

  # Do Prediction on Absent Elements
  lr = LinearRegression()
  lr.fit(X_train,y_train)
  

  data2 = data2.drop(columns=['Age'],axis=1)
  data2['Age']=[max(0,min(100,i)) for i in lr.predict(X_test)]
  
  temp=data1['Age']
  data1 = data1.drop(columns=['Age'],axis=1)
  data1['Age']=temp

  return data1.append(data2)
def impute_fare(dataset):
  if dataset.Fare.isnull().sum()<=0:
    return dataset
  else:
    ok_set = dataset[~dataset.Fare.isnull()]
    to_set = dataset[dataset.Fare.isnull()]

    l=list()
    for i in range(to_set.shape[0]):  
      l.append(ok_set.Fare[(ok_set.Pclass==to_set.iloc[i]['Pclass']) & (ok_set.Parch==to_set.iloc[i]['Parch'])].mean())

    # Structural Re-format (Faster compared to For Loop)
    temp = ok_set['Fare']
    ok_set = ok_set.drop(columns='Fare',axis=1)
    ok_set['Fare'] = temp
    
    to_set=to_set.drop(columns='Fare',axis=1)
    to_set['Fare']=l

    return ok_set.append(to_set)
def impute_cabin(data):
  if data.Cabin.isnull().sum()<=0:
    return data
  else:
    ok_set = data[~data.Cabin.isnull()]
    to_set = data[data.Cabin.isnull()]

    #l=list()
    #for i in range(ok_set.shape[0]):
    # l.append(ok_set.iloc[i]['Cabin'][0])

    l = [ok_set.iloc[i]['Cabin'][0] for i in range(ok_set.shape[0]) ] # Cannot put in next assignment, drop used
    ok_set = ok_set.drop(columns=['Cabin'],axis=1)
    ok_set['Cabin'] = l

    to_set = to_set.drop(columns='Cabin',axis=1)
    to_set['Cabin'] = 'U'

    return ok_set.append(to_set)
def impute_embarked(data):
  data.Embarked.replace('S',1,inplace=True)
  data.Embarked.replace('C',2,inplace=True)
  data.Embarked.replace('Q',3,inplace=True)
  if data.Embarked.isnull().sum()<=0:
    return data
  else:
    ok_data = data[~data.Embarked.isnull()]
    to_data = data[data.Embarked.isnull()]

    lr = LinearRegression() # Using Regression, the mode of Embarked is captured
    lr.fit(ok_data[['Pclass','Fare','Parch']],ok_data['Embarked'])
    
    temp = ok_data['Embarked']
    ok_data = ok_data.drop(columns='Embarked',axis=1)
    ok_data['Embarked'] = temp
    
    to_data = to_data.drop(columns='Embarked',axis=1)
    temp = [int(i) for i in lr.predict(to_data[['Pclass','Fare','Parch']])]
    to_data['Embarked'] = temp

    return ok_data.append(to_data)
# Impute Missing Value 
# ~ Fare
test = impute_fare(test)

# ~ Age
train = get_missing_age(train)
test = get_missing_age(test)

# ~ Cabin
train=impute_cabin(train)
test=impute_cabin(test)

# ~ Embarked
train = impute_embarked(train)
test = impute_embarked(test)
age_group = 3
max_age = max(train.Age)
min_age = min(train.Age)

train['Age_Group'] = [int(i) for i in round((train.Age-min_age)/((max_age-min_age)/age_group),0)+1]
test['Age_Group']  = [int(i) for i in round((test.Age-min_age)/((max_age-min_age)/age_group),0)+1]
train = train.drop(columns='Age')
test = test.drop(columns='Age')
pd.pivot_table(train, values='Fare', index=['Pclass'],columns=['Parch'], aggfunc=np.mean)
def get_feature_count(data,col_names):
  df_all=pd.DataFrame()
  for i in col_names:
    u = data[i].unique()
    temp=pd.DataFrame()
    for j in u:
      m = (data[i]==j).sum()
      temp = temp.append([[j,m]])
    temp['col_name'] = i    
    df_all = df_all.append(temp)

  df_all.columns = ['X','Y','Feature']
  return df_all
df = get_feature_count(train,['Pclass','Sex','Cabin','Embarked','Age_Group','SibSp','Parch'])

fig=px.bar(data_frame=df, x='X',y='Y',color='Y',facet_col='Feature',facet_col_wrap=7,width=1000,height=350)
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show()

testes para fazer funcionar

In [ ]:
start_ngrok()
!python dash/titanic.py